In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import copy
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import mean_squared_error,log_loss, roc_auc_score

sub = pd.read_csv("SampleSubmission.csv")
train  = pd.read_csv("dsn_Train.csv")
test  = pd.read_csv("dsn_Test.csv") 

train_copy = train.copy()
test_copy = test.copy()


train = train.fillna(0)
test = test.fillna(0)

# test.info()

replace1 = {'default_status': {'yes':1, 'no':0}}
replace2 = {'form_field47': {'lending':1, 'charge':0}}

train.replace(replace1, inplace=True)

train.replace(replace2, inplace=True)
test.replace(replace2, inplace=True)

train.set_index('Applicant_ID', inplace=True)
test.set_index('Applicant_ID', inplace=True)

In [67]:
#get list
def get_list(prediction):
    pl = []
    for i in prediction:
        pl.append(i[0])
    return pl

#Balancing function
def make_balanced(train, test):
    cols = [x for x in list(train.columns)]
    cols.pop(-1)
    
    for i in cols:
        train[i] = np.log1p(train[i])
        test[i] = np.log1p(test[i])

#model fitting
def model_fit(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)

    preds = model.predict_proba(X_test)
    preds = preds[:,1:]
    pl = []
    for i in preds:
        pl.append(i[0])

#     print("roc: ",roc_auc_score(y_test,preds))
    return roc_auc_score(y_test,preds)
    
#Scaler function
from sklearn.preprocessing import StandardScaler
def scale(train, test):
    scaler = StandardScaler()

#     scaler.fit(train)

    train_scaled = pd.DataFrame(scaler.fit_transform(train))
    test_scaled = pd.DataFrame(scaler.fit_transform(test))

    train_scaled.index = train.index
    test_scaled.index = test.index
    
    train_scaled.columns = train.columns
    test_scaled.columns = test.columns
    
    return train_scaled, test_scaled

#Creating Prediction List
def predictList(x, model):
    prediction = model.predict_proba(x)
    prediction = prediction[:,1:]

    pl = []
    for i in prediction:
        pl.append(i[0])
    return pl

#model prediction for stacking
def model_predictions(model, X_train, y_train, X_test, test):
    
    #train the model
    model.fit(X_train,y_train)
    
    #storing predictions for train and test
    pred_train=model.predict_proba(X_train)[:,1:]
    pred_test=model.predict_proba(X_test)[:,1:]
    pred_main = model.predict_proba(test)[:,1:]
    return get_list(pred_train), get_list(pred_test), get_list(pred_main)

#Submission
def sub(x, predict_scores, test):
    output = pd.DataFrame({'Applicant_ID': test.index,
                          'default_status': predict_scores})

    output.to_csv(f'submission_{x}.csv', index=False)
    

In [68]:
X = train.drop(['default_status'], axis=1)
y = train[['default_status']]

In [69]:
#scaling
X, test = scale(X, test)

In [105]:
#Using Xgboost algorithm
from xgboost import XGBClassifier

xgb = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.4,
                      subsample = 1,
                      objective='binary:logistic', 
                      n_estimators=1000, 
                      reg_alpha = 0.3,
                      max_depth=9, 
                      gamma=1)

# catboost
from catboost import CatBoostClassifier

cat = CatBoostClassifier()

# light boost
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(
            nthread=6,
            n_estimators=1000,
            learning_rate=0.01,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=9,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=600, max_depth=20, n_jobs=5)

In [106]:
lgb2 = LGBMClassifier(n_estimators=1)

from sklearn.ensemble import VotingClassifier
modelV = VotingClassifier(estimators=[('lgb', lgb), ('cat2', cat), ('xgb', xgb), ('rfc', rfc)], voting='soft')

# StraifiedKFold
from sklearn.model_selection import KFold, StratifiedKFold
splits = 10
skfold = StratifiedKFold(n_splits=splits, random_state=42, shuffle=True)
kfold = KFold(n_splits=splits, shuffle=True, random_state=42)

scores = []
preds = []
i = 1

for train_split, test_split in skfold.split(X, y):
    x_train, x_test, y_train, y_test = X.iloc[train_split], X.iloc[test_split], y['default_status'].iloc[train_split], y['default_status'].iloc[test_split]
    %time score = model_fit(modelV,x_train, y_train, x_test, y_test)   
    pred = get_list(modelV.predict_proba(test)[:,1:])
    scores.append(score)
    preds.append(pred)
    print('Area under curve {} split: '.format(i), score)
    i +=1
 
test_pred = np.mean(preds, axis=0)
print('Final absolute mean error: ', np.mean(scores))

[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=6 will be ignored. Current value: num_threads=-1
Learning rate set to 0.054938
0:	learn: 0.6592023	total: 56.6ms	remaining: 56.6s
1:	learn: 0.6305228	total: 99.1ms	remaining: 49.5s
2:	learn: 0.6044408	total: 145ms	remaining: 48s
3:	learn: 0.5826357	total: 189ms	remaining: 47.2s
4:	learn: 0.5632034	total: 255ms	remaining: 50.8s
5:	learn: 0.5485259	total: 323ms	remaining: 53.5s
6:	learn: 0.5356754	total: 372ms	remaining: 52.8s
7:	learn: 0.5228298	total: 427ms	remaining: 53s
8:	learn: 0.5111757	total: 473ms	remaining: 52.1s
9:	learn: 0.5024900	total: 517ms	remaining: 51.2s
10:	learn: 0.4947115	total: 556ms	remaining: 50s
11:	learn: 0.4876356	total: 609ms	remaining: 50.2s
12:	learn: 0.4810173	total: 663ms	remaining: 50.4s
13:	learn: 0.4757959	total: 712ms	remaining: 50.2s
14:	learn: 0.4700680	total: 755ms	remaining: 49.5s
15:	learn: 0.4656070	total: 811ms	remaining: 49.9s
16:	learn: 0.4611678	total: 863ms	remaining: 49.9s
17:	

160:	learn: 0.3980544	total: 7.23s	remaining: 37.7s
161:	learn: 0.3979474	total: 7.28s	remaining: 37.6s
162:	learn: 0.3978670	total: 7.32s	remaining: 37.6s
163:	learn: 0.3978086	total: 7.35s	remaining: 37.5s
164:	learn: 0.3977410	total: 7.39s	remaining: 37.4s
165:	learn: 0.3976205	total: 7.44s	remaining: 37.4s
166:	learn: 0.3974873	total: 7.48s	remaining: 37.3s
167:	learn: 0.3973764	total: 7.52s	remaining: 37.3s
168:	learn: 0.3972940	total: 7.56s	remaining: 37.2s
169:	learn: 0.3971918	total: 7.61s	remaining: 37.1s
170:	learn: 0.3971089	total: 7.65s	remaining: 37.1s
171:	learn: 0.3970070	total: 7.7s	remaining: 37.1s
172:	learn: 0.3969578	total: 7.73s	remaining: 37s
173:	learn: 0.3968435	total: 7.77s	remaining: 36.9s
174:	learn: 0.3967600	total: 7.82s	remaining: 36.8s
175:	learn: 0.3966237	total: 7.87s	remaining: 36.8s
176:	learn: 0.3965583	total: 7.91s	remaining: 36.8s
177:	learn: 0.3964496	total: 7.95s	remaining: 36.7s
178:	learn: 0.3963321	total: 7.99s	remaining: 36.6s
179:	learn: 0.3

319:	learn: 0.3811793	total: 14s	remaining: 29.7s
320:	learn: 0.3810611	total: 14s	remaining: 29.7s
321:	learn: 0.3809421	total: 14.1s	remaining: 29.6s
322:	learn: 0.3808377	total: 14.1s	remaining: 29.6s
323:	learn: 0.3807582	total: 14.1s	remaining: 29.5s
324:	learn: 0.3806731	total: 14.2s	remaining: 29.5s
325:	learn: 0.3805337	total: 14.2s	remaining: 29.4s
326:	learn: 0.3804391	total: 14.3s	remaining: 29.4s
327:	learn: 0.3803104	total: 14.3s	remaining: 29.3s
328:	learn: 0.3801871	total: 14.4s	remaining: 29.3s
329:	learn: 0.3800943	total: 14.4s	remaining: 29.3s
330:	learn: 0.3799818	total: 14.5s	remaining: 29.2s
331:	learn: 0.3798667	total: 14.5s	remaining: 29.2s
332:	learn: 0.3797820	total: 14.5s	remaining: 29.1s
333:	learn: 0.3796886	total: 14.6s	remaining: 29.1s
334:	learn: 0.3795846	total: 14.6s	remaining: 29s
335:	learn: 0.3794903	total: 14.7s	remaining: 29s
336:	learn: 0.3794038	total: 14.7s	remaining: 28.9s
337:	learn: 0.3793172	total: 14.7s	remaining: 28.9s
338:	learn: 0.379235

481:	learn: 0.3667291	total: 20.7s	remaining: 22.2s
482:	learn: 0.3666228	total: 20.7s	remaining: 22.2s
483:	learn: 0.3665512	total: 20.8s	remaining: 22.2s
484:	learn: 0.3664441	total: 20.8s	remaining: 22.1s
485:	learn: 0.3663607	total: 20.9s	remaining: 22.1s
486:	learn: 0.3662485	total: 20.9s	remaining: 22s
487:	learn: 0.3661478	total: 20.9s	remaining: 22s
488:	learn: 0.3660765	total: 21s	remaining: 21.9s
489:	learn: 0.3659662	total: 21s	remaining: 21.9s
490:	learn: 0.3658937	total: 21.1s	remaining: 21.8s
491:	learn: 0.3658885	total: 21.1s	remaining: 21.8s
492:	learn: 0.3657884	total: 21.1s	remaining: 21.8s
493:	learn: 0.3657213	total: 21.2s	remaining: 21.7s
494:	learn: 0.3656387	total: 21.2s	remaining: 21.7s
495:	learn: 0.3655463	total: 21.3s	remaining: 21.6s
496:	learn: 0.3654752	total: 21.3s	remaining: 21.6s
497:	learn: 0.3653902	total: 21.4s	remaining: 21.5s
498:	learn: 0.3652981	total: 21.4s	remaining: 21.5s
499:	learn: 0.3652208	total: 21.4s	remaining: 21.4s
500:	learn: 0.365149

644:	learn: 0.3536430	total: 27.5s	remaining: 15.1s
645:	learn: 0.3535417	total: 27.6s	remaining: 15.1s
646:	learn: 0.3534397	total: 27.6s	remaining: 15.1s
647:	learn: 0.3533204	total: 27.7s	remaining: 15s
648:	learn: 0.3532448	total: 27.7s	remaining: 15s
649:	learn: 0.3531785	total: 27.7s	remaining: 14.9s
650:	learn: 0.3531744	total: 27.8s	remaining: 14.9s
651:	learn: 0.3531598	total: 27.8s	remaining: 14.8s
652:	learn: 0.3530872	total: 27.9s	remaining: 14.8s
653:	learn: 0.3530001	total: 27.9s	remaining: 14.8s
654:	learn: 0.3529168	total: 27.9s	remaining: 14.7s
655:	learn: 0.3528304	total: 28s	remaining: 14.7s
656:	learn: 0.3527292	total: 28s	remaining: 14.6s
657:	learn: 0.3526650	total: 28.1s	remaining: 14.6s
658:	learn: 0.3525871	total: 28.1s	remaining: 14.5s
659:	learn: 0.3525039	total: 28.2s	remaining: 14.5s
660:	learn: 0.3524103	total: 28.2s	remaining: 14.5s
661:	learn: 0.3523219	total: 28.2s	remaining: 14.4s
662:	learn: 0.3522545	total: 28.3s	remaining: 14.4s
663:	learn: 0.352182

...

295:	learn: 0.3831482	total: 23.1s	remaining: 55s
296:	learn: 0.3830296	total: 23.2s	remaining: 55s
297:	learn: 0.3829437	total: 23.3s	remaining: 54.9s
298:	learn: 0.3828232	total: 23.4s	remaining: 54.9s
299:	learn: 0.3826895	total: 23.5s	remaining: 54.8s
300:	learn: 0.3826038	total: 23.6s	remaining: 54.7s
301:	learn: 0.3825169	total: 23.6s	remaining: 54.6s
302:	learn: 0.3823917	total: 23.7s	remaining: 54.6s
303:	learn: 0.3822464	total: 23.8s	remaining: 54.5s
304:	learn: 0.3821383	total: 23.9s	remaining: 54.5s
305:	learn: 0.3820681	total: 24s	remaining: 54.4s
306:	learn: 0.3819523	total: 24.1s	remaining: 54.3s
307:	learn: 0.3818627	total: 24.1s	remaining: 54.2s
308:	learn: 0.3817545	total: 24.2s	remaining: 54.2s
309:	learn: 0.3816507	total: 24.3s	remaining: 54.1s
310:	learn: 0.3815326	total: 24.4s	remaining: 54.1s
311:	learn: 0.3814191	total: 24.5s	remaining: 54s
312:	learn: 0.3813339	total: 24.6s	remaining: 54s
313:	learn: 0.3812338	total: 24.7s	remaining: 54s
314:	learn: 0.3811067	to

455:	learn: 0.3678713	total: 38s	remaining: 45.4s
456:	learn: 0.3678117	total: 38.1s	remaining: 45.3s
457:	learn: 0.3677653	total: 38.2s	remaining: 45.2s
458:	learn: 0.3677611	total: 38.3s	remaining: 45.1s
459:	learn: 0.3676570	total: 38.4s	remaining: 45s
460:	learn: 0.3675407	total: 38.4s	remaining: 44.9s
461:	learn: 0.3674413	total: 38.5s	remaining: 44.8s
462:	learn: 0.3674367	total: 38.6s	remaining: 44.8s
463:	learn: 0.3673280	total: 38.7s	remaining: 44.7s
464:	learn: 0.3672322	total: 38.8s	remaining: 44.6s
465:	learn: 0.3671116	total: 38.9s	remaining: 44.5s
466:	learn: 0.3669945	total: 38.9s	remaining: 44.4s
467:	learn: 0.3669020	total: 39s	remaining: 44.4s
468:	learn: 0.3668111	total: 39.1s	remaining: 44.3s
469:	learn: 0.3667250	total: 39.2s	remaining: 44.2s
470:	learn: 0.3666541	total: 39.3s	remaining: 44.1s
471:	learn: 0.3665564	total: 39.4s	remaining: 44s
472:	learn: 0.3664507	total: 39.5s	remaining: 44s
473:	learn: 0.3663453	total: 39.6s	remaining: 43.9s
474:	learn: 0.3662643	

614:	learn: 0.3549121	total: 51.1s	remaining: 32s
615:	learn: 0.3548582	total: 51.2s	remaining: 31.9s
616:	learn: 0.3547813	total: 51.3s	remaining: 31.9s
617:	learn: 0.3547113	total: 51.4s	remaining: 31.8s
618:	learn: 0.3546189	total: 51.5s	remaining: 31.7s
619:	learn: 0.3545213	total: 51.6s	remaining: 31.6s
620:	learn: 0.3545189	total: 51.6s	remaining: 31.5s
621:	learn: 0.3544491	total: 51.7s	remaining: 31.4s
622:	learn: 0.3543832	total: 51.8s	remaining: 31.3s
623:	learn: 0.3543202	total: 51.9s	remaining: 31.2s
624:	learn: 0.3542438	total: 51.9s	remaining: 31.2s
625:	learn: 0.3541824	total: 52s	remaining: 31.1s
626:	learn: 0.3541002	total: 52.1s	remaining: 31s
627:	learn: 0.3540021	total: 52.2s	remaining: 30.9s
628:	learn: 0.3539542	total: 52.3s	remaining: 30.8s
629:	learn: 0.3538658	total: 52.4s	remaining: 30.8s
630:	learn: 0.3537809	total: 52.4s	remaining: 30.7s
631:	learn: 0.3536768	total: 52.5s	remaining: 30.6s
632:	learn: 0.3536132	total: 52.6s	remaining: 30.5s
633:	learn: 0.3535

774:	learn: 0.3432743	total: 1m 4s	remaining: 18.6s
775:	learn: 0.3432059	total: 1m 4s	remaining: 18.5s
776:	learn: 0.3431425	total: 1m 4s	remaining: 18.4s
777:	learn: 0.3430706	total: 1m 4s	remaining: 18.3s
778:	learn: 0.3429931	total: 1m 4s	remaining: 18.2s
779:	learn: 0.3429181	total: 1m 4s	remaining: 18.2s
780:	learn: 0.3429164	total: 1m 4s	remaining: 18.1s
781:	learn: 0.3428651	total: 1m 4s	remaining: 18s
782:	learn: 0.3427924	total: 1m 4s	remaining: 17.9s
783:	learn: 0.3427153	total: 1m 4s	remaining: 17.8s
784:	learn: 0.3426374	total: 1m 4s	remaining: 17.7s
785:	learn: 0.3425668	total: 1m 4s	remaining: 17.6s
786:	learn: 0.3424650	total: 1m 4s	remaining: 17.6s
787:	learn: 0.3423943	total: 1m 4s	remaining: 17.5s
788:	learn: 0.3423097	total: 1m 5s	remaining: 17.4s
789:	learn: 0.3422550	total: 1m 5s	remaining: 17.3s
790:	learn: 0.3421834	total: 1m 5s	remaining: 17.2s
791:	learn: 0.3421019	total: 1m 5s	remaining: 17.1s
792:	learn: 0.3420304	total: 1m 5s	remaining: 17.1s
793:	learn: 0.

932:	learn: 0.3326035	total: 1m 19s	remaining: 5.7s
933:	learn: 0.3325336	total: 1m 19s	remaining: 5.61s
934:	learn: 0.3324455	total: 1m 19s	remaining: 5.53s
935:	learn: 0.3323428	total: 1m 19s	remaining: 5.45s
936:	learn: 0.3322908	total: 1m 19s	remaining: 5.36s
937:	learn: 0.3322386	total: 1m 19s	remaining: 5.28s
938:	learn: 0.3321227	total: 1m 20s	remaining: 5.2s
939:	learn: 0.3320682	total: 1m 20s	remaining: 5.11s
940:	learn: 0.3320136	total: 1m 20s	remaining: 5.03s
941:	learn: 0.3319585	total: 1m 20s	remaining: 4.95s
942:	learn: 0.3318847	total: 1m 20s	remaining: 4.86s
943:	learn: 0.3318330	total: 1m 20s	remaining: 4.78s
944:	learn: 0.3317908	total: 1m 20s	remaining: 4.7s
945:	learn: 0.3317116	total: 1m 20s	remaining: 4.61s
946:	learn: 0.3316453	total: 1m 20s	remaining: 4.53s
947:	learn: 0.3315567	total: 1m 20s	remaining: 4.44s
948:	learn: 0.3314699	total: 1m 21s	remaining: 4.36s
949:	learn: 0.3313775	total: 1m 21s	remaining: 4.27s
950:	learn: 0.3312762	total: 1m 21s	remaining: 4.

In [81]:
test_pred = np.mean(preds, axis=0)

In [107]:
sub('kfold10xV', test_pred, test)

In [110]:
test = test.reset_index()

In [111]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,-0.125237,-0.209072,4.168049,-0.251646,-0.190675,-0.047188,-0.368115,-0.442573,-0.508740,...,-0.633437,1.124127,-0.864421,1.152485,-0.155399,-0.221065,-0.742759,-0.127300,-0.583318,-0.091929
1,Apcnt_1000048,-0.061803,0.933619,0.867035,2.406447,-0.190675,-0.368355,0.236605,-0.543792,-0.464117,...,-0.287461,0.355756,-0.388371,0.448859,5.203188,-0.221065,1.346332,-0.127300,-0.304812,-0.091929
2,Apcnt_1000052,-4.401717,-0.300131,-0.455468,-0.251646,-0.190675,-0.368355,-0.394928,-0.557217,-0.533630,...,-0.633437,-0.873648,-1.023104,-2.032208,-0.155399,-0.221065,-0.742759,-0.127300,-0.583318,-0.091929
3,Apcnt_1000076,-0.130523,-0.287776,0.795270,-0.251646,-0.190675,-0.368355,-0.364577,-0.427443,-0.454251,...,-0.633437,1.346094,-0.705738,-0.184194,-0.155399,-0.221065,-0.742759,-0.127239,-0.148270,-0.091929
4,Apcnt_1000080,0.178716,1.949048,-0.103572,0.490456,-0.190675,-0.353132,-0.321219,-0.242053,-0.171794,...,-0.633437,-0.306893,2.626613,0.860640,-0.155399,-0.221065,1.346332,-0.126424,0.319933,-0.091929


In [112]:
pd.DataFrame({"Applicant_ID": test["Applicant_ID"], "default_status": test_pred}).to_csv("SampleSubmission.csv", index = False)